In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score
# from tensorflow.keras.callbacks import LearningRateScheduler
# from tensorflow.keras.utils import to_categorical  # Import to_categorical from keras.utils
import matplotlib.pyplot as plt
# from tensorflow.keras.regularizers import l1_l2,l1,l2
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
import itertools
import random

In [18]:
def configure_model(inp_shape,num_classes):
    # Build the deep learning model
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='leaky_relu',input_shape=inp_shape),
        tf.keras.layers.Dense(32, activation='leaky_relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    # # Define a custom learning rate schedule function
    # def lr_schedule(epoch):
    #     if epoch < 10:
    #         return 0.001
    #     elif epoch < 20:
    #         return 0.0001
    #     else:
    #         return 0.00001

    return model


In [19]:
def model_train_and_test(model,lr,X_train,y_train,X_test,y_test):
    
    custom_optimizer = Adam(learning_rate=lr)
    # Compile the model with sparse-categorical cross-entropy loss
    model.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.25, verbose=0)
    y_pred = model.predict(X_test)
    y_pred_class = y_pred.argmax(axis=1)
    accuracy = accuracy_score(y_test, y_pred_class)

    return accuracy


In [20]:
#for reading dataset-1
# Load your CSV data1 using pandas
data1 = pd.read_csv('classData.csv')

# Combine the values of the first 4 columns into a new column
data1['Combination'] = data1.apply(lambda row: ''.join(map(str, row[:4])), axis=1)

# Find unique combinations and assign label numbers
unique_combinations = data1['Combination'].unique()
label_mapping = {combination: label for label, combination in enumerate(unique_combinations)}

# Add a new column with the label numbers
data1['Label'] = data1['Combination'].map(label_mapping)

X1 = data1.iloc[:, 4:10].values  # Features
y1 = data1.iloc[:, 11:].values  # Labels

# Standardize the data
scaler = StandardScaler()
X1 = scaler.fit_transform(X1)

# One-hot encode the labels
# y1_encoded = to_categorical(y1, num_classes=6)
num_classes=6
y1_encoded = y1

In [21]:
# Genetic Algorithm Configuration
population_size = 10
generations = 10
mutation_rate = 0.1
learning_rate_range = (0.0001, 0.1)

# Initialize the population with random learning rates
population = [random.uniform(*learning_rate_range) for _ in range(population_size)]
fitness_scores = [0] * population_size  # Initialize with zeros

In [22]:
#Compiling and testing model for dataset-1

#Splitting dataset-1 into train and test
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1_encoded, test_size=0.2, random_state=42)

input_shape1=(X_train1.shape[1],)
num_classes=6
model1=configure_model(input_shape1,num_classes)


In [23]:
def generate_population(model1,population,X_train,y_train,X_test,y_test):
    for generation in range(generations):
        # Evaluate the fitness of each individual in the population
        for i, learning_rate in enumerate(population):
            fitness_scores[i] = model_train_and_test(model1,learning_rate,X_train,y_train,X_test,y_test)

        # Select the top performers (elite)
        elite_count = int(population_size * 0.2)
        elite_indices = np.argsort(fitness_scores)[-elite_count:]

        # Create a new population with elite individuals and mutated individuals
        new_population = [population[i] for i in elite_indices]

        # Perform mutation
        for i in range(population_size - elite_count):
            if random.random() < mutation_rate:
                # Mutate by generating a new random learning rate
                new_population.append(random.uniform(*learning_rate_range))
            else:
                # Select a parent from the elite individuals
                parent_index = random.choice(elite_indices)
                new_population.append(population[parent_index])

        population = new_population
    
    return population


In [24]:

final_population1=generate_population(model1,population,X_train1,y_train1,X_test1,y_test1)

# Find the best learning rate in the final population
best_learning_rate1 = final_population1[np.argmax(fitness_scores)]

print(f"Best Learning Rate: of dataset-1 is {best_learning_rate1:.5f}")

#Corresponding accuracy of best learning rate model is
best_accuracy1 = max(fitness_scores)

print(f"Best Accuracy: of dataset-1 is {best_accuracy1:.5f}")

50/50 [==============================] - 0s 833us/step
Best Learning Rate: of dataset-1 is 0.00650
Best Accuracy: of dataset-1 is 0.87921


In [25]:
#Dataset preprocessing for dataset-2

# Load your CSV data using pandas
data2 = pd.read_csv('cwru_bearing.csv')
import pandas as pd

# Step 1: Identify unique labels in the last column
unique_labels = data2['fault'].unique()

# Step 2: Create a mapping from labels to numbers
label_to_number = {label: index for index, label in enumerate(unique_labels)}

# Step 3: Replace labels with corresponding numbers
data2['fault'] = data2['fault'].map(label_to_number)

# Now, 'Y' contains numeric labels from 1 to 11

# You can save the modified DataFrame to a new CSV file if needed
# data2.to_csv('cwru_bearing_numeric_labels.csv', index=False)
# Standardize the data

X2 = data2.iloc[:, 0:9].values  # Features
y2 = data2.iloc[:, 9].values  # Labels

# Standardize the data
scaler = StandardScaler()
X2 = scaler.fit_transform(X2)


In [27]:
#Compiling and testing model for dataset-2

#Splitting dataset-2 into train and test
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)

input_shape2=(X_train2.shape[1],)
num_classes=10
model2=configure_model(input_shape2,num_classes)

In [28]:
final_population2=generate_population(model2,population,X_train2,y_train2,X_test2,y_test2)

# Find the best learning rate in the final population
best_learning_rate2 = final_population2[np.argmax(fitness_scores)]

print(f"Best Learning Rate: of dataset-2 is {best_learning_rate2:.5f}")

#Corresponding accuracy of best learning rate model is
best_accuracy2 = max(fitness_scores)

print(f"Best Accuracy: of dataset-2 is {best_accuracy2:.5f}")


15/15 [==============================] - 0s 890us/step
Best Learning Rate: of dataset-2 is 0.01690
Best Accuracy: of dataset-2 is 0.95217
